<a href="https://colab.research.google.com/github/AbhiMalappa/NLP/blob/main/helpdesk_BERT_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from torch import nn
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline
import torch


In [ ]:
# funct to get predictions
model_ckpt = "drive/MyDrive/Colab Notebooks/NLP/trained_model/Finetune_BERT_for_helpdesk_issue_classification"
model_ckpt = "drive/MyDrive/Colab Notebooks/NLP/trained_model/V2_Finetune_BERT_for_helpdesk_issue_classification_7_class"

def get_pred(text):

  device = torch.device("cuda")

  #read model
  model = AutoModelForSequenceClassification.from_pretrained(model_ckpt).to(device)

  # from model config get labels names
  labels = model.config.id2label.values()
  labels = list(labels)

  #tokenize input
  tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
  input_encoded = tokenizer(text, return_tensors='pt').to(device)

  # get model pred on tokenized input
  with torch.no_grad():
    outputs = model(**input_encoded) # output is a torch tensor with logit values on gpu

  #output logits
  logits = outputs.logits[0] # get logit values
  #output probs
  probabilities = nn.functional.softmax(logits, dim=-1) # convert logit to prob

  # return a dic with labels and probs
  probabilities = probabilities.cpu().detach().numpy() # convert to numpy on cpu
  result = {labels[i]: probabilities[i] for i in range(len(labels))}

  return result, #probabilities

def pipeline_clf(text):
  device = torch.device("cuda")
  clf = pipeline('text-classification',model = model_ckpt)
  pipeline_clf_label = clf(text)[0]['label']
  pipeline_clf_score = clf(text)[0]['score']
  return pipeline_clf_label, pipeline_clf_score


### Predict on new data

In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv('drive/MyDrive/Colab Notebooks/training_text_calssifiaction.csv')
df = df.sample(3)

# call prediction
df['pred_label'] = df['string_value'].apply(lambda x: pipeline_clf(x)[0])
df['pred_label_score'] = df['string_value'].apply(lambda x: pipeline_clf(x)[1])
df['pred_val_all_class'] = df['string_value'].apply(lambda x: get_pred(x))

df.head(2)

### Predict on text input

In [ ]:
text = [
    " agent went home early "
    ]

In [ ]:
sentement_obj(text)
#print(f' Top pred - {pipeline_clf(text)}\n')

get_pred(text)

[{'label': 'neutral', 'score': 0.8309993743896484}]

({'agent_unprofessional': 0.058254667,
  'long_holdtime': 0.5411721,
  'multiple_contacts_made': 0.14343858,
  'negative_feedback': 0.11775059,
  'no_resolution': 0.044664945,
  'positive_feedback': 0.083976366,
  'service_portal': 0.0107428245},)